In [1]:
import random
import pandas as pd
from datetime import datetime, timedelta

In [13]:
# Создаем список товаров
num_products = 9
products = [f'Product_{i}' for i in range(1, num_products + 1)]

# Задаем диапазон дат
start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 12, 31)

# Инициализируем начальный уровень инвентаря и параметры заказов для каждого товара
initial_inventory = {product: random.randint(100, 200) for product in products}
order_parameters = {product: random.randint(8, 15) * 10 for product in products}

# Добавляем переменные для времени доставки, цены доставки, цены хранения и цены продажи
delivery_times = {product: random.randint(2, 5) for product in products}  # Пример времени доставки в днях
storage_prices = {product: round(random.uniform(0.1, 1.0), 2) for product in products}  # Пример цены хранения
delivery_prices = {product: round(random.uniform(2, 10), 2) for product in products}  # Пример цены доставки
sale_prices = {product: int(random.uniform(20, 99)) for product in products}  # Пример цены продажи

In [9]:
# Добавляем переменные для сезонных факторов
season_factors = {1: 1.0, 2: 1.1, 3: 1.1, 4: 1.2, 5: 1.2, 6: 1.3, 7: 1.4, 8: 1.4, 9: 1.5, 10: 1.6, 11: 1.6, 12: 1.7}

In [16]:
# Генерируем данные о продажах, инвентаре, заказах, ценах продаж и сезонных факторах
data = []
for product in products:
    current_date = start_date
    inventory = initial_inventory[product]
    order_placed = False  # Флаг, указывающий, был ли уже размещён заказ
    delivery_date = None  # Инициализация переменной delivery_date
    unrealized_demand = 0 # Инициализируем unrealized_demand перед циклом
    
    # Используем фиксированные значения для storage_price и sale_price
    storage_price = storage_prices[product]
    sale_price = sale_prices[product]
    
    while current_date <= end_date:
        month = current_date.month
        seasonal_factor = season_factors[month]
        sales = round(random.randint(15, 20) * seasonal_factor) # случайное количество продаж
        
        if current_date.weekday() in [5, 6]:
            # Если текущий день выходной (5 - суббота, 6 - воскресенье),
            # увеличиваем количество продаж в выходные
            sales += random.randint(2, 5)
            
        if delivery_date == current_date:
            # Увеличиваем инвентарь в день прихода:
            inventory += order_quantity
            order_placed = False  # Сбрасываем флаг размещения заказа после увеличения инвентаря в день доставки
            
        inventory -= sales
        if inventory < 0:
            unrealized_demand = abs(inventory)
            sales = sales - unrealized_demand
            inventory = 0
        
            
        if inventory < order_parameters[product] and not order_placed:
            # Если инвентарь меньше порога, делаем заказ
            order_quantity = random.randint(6, 20) * 10
            order = order_quantity
            delivery_time = delivery_times[product]
            delivery_price = delivery_prices[product]
            storage_price = storage_prices[product]
            sale_price = sale_prices[product]
                                
            # Рассчитываем дату доставки
            delivery_date = current_date + timedelta(days=delivery_time)
            order_placed = True
                        
        else:
            order = 0
            delivery_time = 0
            delivery_price = 0
                             
        data.append([current_date, product, sales, order, inventory, unrealized_demand, delivery_time, delivery_price, storage_price, sale_price])
        current_date += timedelta(days=1)

In [18]:
# Создаем DataFrame для данных о продажах, инвентаре, заказах, дополнительных параметрах, сезонных факторах и дате заполнения инвентаря
columns = ['Date', 'Product', 'Sales', 'Orders', 'Inventory', 'UnrealizedDemand',
           'DeliveryTime', 'DeliveryPrice', 'StoragePrice', 'SalePrice']
sales_inventory_orders_data = pd.DataFrame(data, columns=columns)

# Сохраняем данные в CSV файл
sales_inventory_orders_data.to_csv('sales_inventory_orders_data.csv', index=False)